In [2]:
# Import necessary packages
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
list_of_genres = ['children', 'fantasy', 'magical', 'mystery', 'action', 'crime', 'drama', 'science fiction', 'adventure', 'biography', 'comedy', 'thriller', 'musical', 'western', 'romance', 'animation', 'war', 'horror', 'sport', 'political', 'saga', ' satire', 'social']

In [4]:
#Works
def get_IMDB_names(url):
    rn = requests.get(url)
    soup = BeautifulSoup(rn.text, 'html.parser')
    movie_containers = soup.find_all('div', class_ = 'lister-item mode-advanced')
    names = []
    for container in movie_containers:
# Extract:
# The name
        name = container.select('img')[0]['alt']
        names.append(name)
    return names

In [5]:
def get_IMDB_years(url):
    ry = requests.get(url)
    soup = BeautifulSoup(ry.text, 'html.parser')
    movie_containers = soup.find_all('div', class_ = 'lister-item mode-advanced')
    years = []
    for container in movie_containers:
        # The year
        year = container.find('span', class_ = 'lister-item-year').text
        year = str(year)
        years.append(year)
        year = str(year)
        year = year[1:5]
    return years

In [6]:
def get_IMDB_genres(url):
    rg = requests.get(url)
    soup = BeautifulSoup(rg.text, 'html.parser')
    movie_containers = soup.find_all('div', class_ = 'lister-item mode-advanced')
    genres =[] 
    for container in movie_containers:
        genre_container = container.find('p')
        genre = genre_container.find('span', class_='genre')
        #genre = genre.text
        #genre = str(genre)
        #genre = genre[1:]
        genres.append(genre)
    return genres

In [8]:
#function that will get the violence grade for a year
def get_IMDB_table(year):
    url = 'https://www.imdb.com/search/title/?release_date='+ str(year) + '-01-01,' + str(year) + '-12-31&user_rating=1.0,10.0&countries=us&sort=user_rating,desc&count=250'
    names = get_IMDB_names(url)
    #years = get_IMDB_years(url)
    genres = get_IMDB_genres(url)
    initial_df = pd.DataFrame({'name': names, 'genre': genres})
    #turn BS elements into strings
    initial_df['genre']=initial_df['genre'].map(lambda x: str(x))
    # add a year column             
    years = [year]*len(initial_df)
    initial_df['year'] = years
    #create a list with all the genres from the "Genres" column and lower the case of each element
    list_year_1 = initial_df['genre'].to_list()
    lower_list = [x.lower() for x in list_year_1]
    #separate the genres where a movie has two or more genres
    list_final = [x.split() for x in lower_list]
    list_year = []
    for sublist in list_final:
        for item in sublist:
            if item == '<span':
                continue
            elif item == 'class="genre">':
                continue
            elif item == '</span>':
                continue
            else:
                list_year.append(item)
    #some genres have comma attached, take off comma
            list_year_no_comma = []
            for item in list_year:
                if ',' in item:
                        list_year_no_comma.append(item[:-1])
                else:
                            list_year_no_comma.append(item)
    #return list_year_no_comma
    # Create a dictionary key- genres, value = number of movies of a certain genre
    genre_counts = {}
# Iterate through genre list
    for genre in list_year_no_comma:
# Update word counts - keep only genres in the general list of genres
        if genre in list_of_genres:
            genre_counts[genre] = genre_counts.get(genre, 0) + 1 #Get previous entry, update by 1
        else:
            continue
#Set 'Genres' as df index
    df_to_plot = pd.DataFrame(genre_counts.items(), columns=['Genres', 'Year'])
    #return df_to_plot
    df_to_plot.set_index('Genres')
    return df_to_plot
# Join the the 2 Dataframes on 'Genres' column and replace NANs with mean value.
    #Zeitgeist_df = war_grades_df.join(df_to_plot.set_index('Genres'), on='Genres')
    #Zeitgeist_df = Zeitgeist_df.fillna(5)
#Create a column year_war_grade 
    #Zeitgeist_df['year_war_grade'] = Zeitgeist_df['Grade']*Zeitgeist_df['Year']
# Add a year column
    #year = year
    #years_final = [year]*len(Zeitgeist_df)
    #Zeitgeist_df['Year'] = years_final
    #del Zeitgeist_df['Grade']
           
    #return Zeitgeist_df
#Zeitgeist1980 = get_table_after_1996(1980)
#Zeitgeist1980
get_IMDB_table(1980)

,Genres,Year
0,thriller,2
1,horror,2
2,adventure,27
3,drama,87
4,comedy,108
5,action,18
6,crime,7
7,mystery,3
8,sport,3
9,romance,16


In [ ]:
pieces = []
for year in [*range(1980, 2020,1)]:
    df_year = get_table(year)
    pieces.append(df_year)
df_all = pd.concat(pieces)
df_all
df_all.to_csv('IMDB_genres_1980_2019.csv', index=False, header= True)    


In [ ]:
grouped = df_all.groupby(['Year'])['year_war_grade'].sum()
grouped.to_csv('grouped_war_grade_IMDB.csv', index='Year', header=True)

In [100]:
#Works
#function that will create an url when you input the year
def gen_url(year):
    url = 'https://www.imdb.com/search/title/?release_date='+ str(year) + '-01-01,' + str(year) + '-12-31&user_rating=1.0,10.0&countries=us&sort=user_rating,desc&count=250'
    return url

In [101]:
x = gen_url(1980)

In [104]:
df = get_IMDB_table(x)
df.head()
type(df.genre[0])


str

In [95]:
#turn objects in genre column into strings
#df.genre[0] = df.genre[0].text
df['genre']=df['genre'].map(lambda x: str(x))
#df['genre']= df['genre'].map(lambda x: x[20:-7])
df['genre'].head()
#df.genre[0]=df.genre[0][20:-7]

0       <span class="genre">\nMusic            </span>
1    <span class="genre">\nShort, Thriller         ...
2    <span class="genre">\nShort, Horror           ...
3       <span class="genre">\nMusic            </span>
4    <span class="genre">\nAdventure, Drama, Family...
Name: genre, dtype: object

#Function that will scrape a wikipedia page with a list of movies by year, add the year as a new column and return 
# a data frame.

def get_table(year):
    url = 'https://www.imdb.com/search/title/?release_date='+ str(year) + '-01-01,' + str(year) + '-12-31&user_rating=1.0,10.0&countries=us&sort=user_rating,desc&count=250'
    df = get_IMDB_table(url)
    return df.head()
# add a year column             
    #years = [year]*len(df)
    #df['Year'] = years
#create a list with all the genres from the "Genres" column and lower the case of each element
    #list_year_1 = df['genres'].to_list()
    #lower_list = [x.lower() for x in list_year_1]
#separate the genres where a movie has two or more genres
    #list_final = [x.split() for x in lower_list]
    #list_year = []
    #for sublist in list_final:
        #for item in sublist:
            #list_year.append(item)  
#some genres have comma attached, take off comma
    #list_year_no_comma = []
    #for item in list_year:
        #if ',' in item:
         #               list_year_no_comma.append(item[:-1])
        #else:
        #                    list_year_no_comma.append(item)
#print(list_year_no_comma)
# Create a dictionary key- genres, value = number of movies of a certain genre
        #genre_counts = {}
# Iterate through genre list
        #for genre in list_year_no_comma:
# Update word counts - keep only genres in the general list of genres
            #if genre in list_of_genres:
                #genre_counts[genre] = genre_counts.get(genre, 0) + 1 #Get previous entry, update by 1
            #else:
                #continue
#Set 'Genres' as df index
        #df_to_plot = pd.DataFrame(genre_counts.items(), columns=['Genres', 'year_1999'])
        #df_to_plot.set_index('Genres')
# Join the the 2 Dataframes on 'Genres' column and replace NANs with mean value.
        #Zeitgeist_df1 = war_grades_df.join(df_to_plot.set_index('Genres'), on='Genres')
        #Zeitgeist_df1 = Zeitgeist_df1.fillna(5)
#Create a column year_war_grade 
        #Zeitgeist_df1['year_war_grade'] = Zeitgeist_df1['Grade']*Zeitgeist_df1['year_1999']
            #Zeitgeist_df1
# Add a year column
            #year = 1999
        #years_final = [year]*len(Zeitgeist_df1)
        #Zeitgeist_df1['Year'] = years_final
           
    #return Zeitgeist_df1
#Zeitgeist1981 = get_table_after_1996(1981)
#Zeitgeist1981
get_table(1981)

In [11]:
pieces = []
for year in [*range(1999, 2004,1)]:
    url = 'http://en.wikipedia.org/wiki/List_of_American_films_of_'+ str(year)
    df = get_table_after_1996(year)
    pieces.append(df)



In [12]:
df_all = pd.concat(pieces)
df_all
df_all.to_csv('9_11.csv', index=False, header= True)

In [13]:
grouped = df_all.groupby(['Year'])['year_war_grade'].sum()
grouped.to_csv('grouped.csv', index='Year', header=True)

In [14]:
grouped



Year
1999    1375.0
2000    1279.0
2001    1364.0
2002    1495.0
2003    1547.0
Name: year_war_grade, dtype: float64